In [ ]:
import azureml
from IPython.display import display, Markdown
from azureml.core import  Experiment, ScriptRunConfig, Workspace, RunConfiguration
from azureml.core.dataset import Dataset
from azureml.core.environment import Environment
from azureml.core.runconfig import DockerConfiguration


from model_drift import settings, helpers
# check core SDK version number
print("Azure ML SDK Version: ", azureml.core.VERSION)

In [ ]:
# Connect to workspace
ws = Workspace.from_config(settings.AZUREML_CONFIG)

In [ ]:
# Name experiement
input_dataset_name="padchest"
experiment_name = 'generate-data-score'
model_name = "classifier-padchest-trained-chx-frontal-lime_train_07pvtv2y-047.ckpt"
env_name = "monitoring"

compute_target = "nc24-uswest2"

#Experiment
exp = Experiment(workspace=ws, name=experiment_name)

#Environment
environment_file = settings.CONDA_ENVIRONMENT_FILE
project_dir = settings.SRC_DIR
pytorch_env = Environment.from_conda_specification(env_name, file_path =str(environment_file))
pytorch_env.register(workspace=ws)
build = pytorch_env.build(workspace=ws)
pytorch_env.environment_variables["RSLEX_DIRECT_VOLUME_MOUNT"] = "True"

# Run Configuration
run_config = RunConfiguration()
run_config.environment_variables["RSLEX_DIRECT_VOLUME_MOUNT"] = "True"
run_config.environment = pytorch_env
run_config.docker = DockerConfiguration(use_docker=True, shm_size="100G")

# Note: COMMON RUNTIME has a bug where print statements sometimes, disappear.
# Set this flag as a workaround to use the legacy runtime.
run_config.environment_variables["AZUREML_COMPUTE_USE_COMMON_RUNTIME"] = "false"

# Input Dataset
dataset = Dataset.get_by_name(ws, name=input_dataset_name)

args = {
'run_azure': 1,
 'model': model_name,
 'data_folder': dataset.as_named_input('dataset').as_mount(),
 'progress_bar_refresh_rate': 25,
 'log_every_n_steps': 25,
 'flush_logs_every_n_steps': 25,
 'batch_size': 32,
 'accelerator': 'ddp',
 'output_dir': './outputs/',
 'dataset': input_dataset_name,
 }

config = ScriptRunConfig(
    source_directory = str(project_dir), 
    script = "scripts/finetune/score.py",
    arguments=helpers.argsdict2list(args),
)

for param, value in args.items():
    print(f" {param}: {value}")

run_config.target = compute_target
config.run_config = run_config

run = exp.submit(config)
display(Markdown(f"""
- Experiment: [{run.experiment.name}]({run.experiment.get_portal_url()})
- Run: [{run.display_name}]({run.get_portal_url()})
- Target: {config.run_config.target}
"""))

In [ ]:
run.wait_for_completion()
output_file_name = settings.RESULTS_DIR.joinpath("classifier", 'padchest-finetuned-chx-frontalonly', 'frontal_only', 'preds.jsonl')
run.download_file("outputs/preds.jsonl", output_file_path=output_file_name)